# Αναγνώριση ονοματισμένων ονοτήτων

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
from spacy.scorer import Scorer



def data_preparation_for_NER(data_filename, test_size=0.25):
    """
    Given the file that contains annotated data for NER
    splits the dataset to train set and test set.

    Parameters
    ----------
    data_filename : str
        the path to the file that contains the annotated
        data for NER
        
    test_size : float
        the test_size that is given as a parameter to
        train_test_split function of scikit-learn

    Returns
    -------
    tuple
        the tuple of train set and test set
    """
    df_texts = pd.read_csv(data_filename, index_col=None, converters={'entities': eval})
    df_texts['text_entities'] = list(zip(df_texts.text, df_texts.entities))
    data = [(text, {'entities': entities}) for (text, entities) in df_texts['text_entities']]
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=10)
    return train_data, test_data



def train_ner(labels, train_set, model=None, new_model_name="", output_dir=None, n_iter=30):
    """
    Sets up the pipeline and named entity recognizer
    and trains the NER for new entities.

    Parameters
    ----------
    labels : list
        the labels (strings) of new entities 
        
    train_set : list
        the observations of the train set
        
    model : module
        existing SpaCy model
        
    new_model_name : str
        the name of the new model
        
    output_dir: str
        the directory of the output file where the new
        model will be saved
        
    n_iter : int
        the number of the iterations in the training process
    """
    random.seed(0)
    if model is not None:
        nlp = model.load()  # load existing spaCy model
    else:
        nlp = spacy.blank("el")  # create blank model in greek
    # add entity recognizer to model if it's not in the pipeline
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe("ner")

    for label in labels:
        ner.add_label(label)  # add new entity labels to entity recognizer
    
    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.resume_training()
    move_names = list(ner.move_names)
    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    # only train NER
    with nlp.disable_pipes(*other_pipes), warnings.catch_warnings():
        warnings.filterwarnings("once", category=UserWarning, module='spacy')

        sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        for itn in range(n_iter):
            random.shuffle(train_set)
            batches = minibatch(train_set, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
            print("Losses in iteration: " + str(itn + 1), losses)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta["name"] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        
        

def evaluate_ner(output_dir, test_set, print_entities=False):
    """
    Tests the new ner model and scores its performance.

    Parameters
    ----------
    output_dir: str
        the directory of the output file where the new
        model was saved
        
    test_set : list
        the observations of the test set
        
    print_entities : boolean
        if True the entities detected for each text
        in the test set will be printed
        
    Returns
    -------
    dict
        all the metrics ant their scores that measure
        the performance of the new NLP model
    """
    ner_model = spacy.load(output_dir)
    scorer = Scorer()
    # test ner
    for (input_, dict_entities) in test_set:        
        [annot] = dict_entities.values()
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot)
        pred_value = ner_model(input_)
        # precision, recall, f-score for ner etc.
        scorer.score(pred_value, gold)
        if print_entities:
            print("Entities in '%s'" % input_[:70])
            for ent in pred_value.ents:
                print(ent.label_, ent.text)
    return scorer.scores

In [2]:
train_data, test_data = data_preparation_for_NER('../data/votana_total_texts_annotated.csv', 0.25)
print('Training...')
train_ner(['ORGAN', 'PLANT'], train_data, new_model_name='ethnopharmacology_ner', output_dir='./ner_model/', n_iter=30)
print('Testing...')
scores = evaluate_ner('./ner_model/', test_data, True)
print(scores)

Training...
Losses in iteration: 1 {'ner': 28570.548088461135}
Losses in iteration: 2 {'ner': 9805.207960022475}
Losses in iteration: 3 {'ner': 6614.338564360779}
Losses in iteration: 4 {'ner': 4707.225389505464}
Losses in iteration: 5 {'ner': 3780.273178118522}
Losses in iteration: 6 {'ner': 3100.5620462014276}
Losses in iteration: 7 {'ner': 2499.1079957860065}
Losses in iteration: 8 {'ner': 2131.7766989105457}
Losses in iteration: 9 {'ner': 2012.188945840103}
Losses in iteration: 10 {'ner': 1654.2847715381367}
Losses in iteration: 11 {'ner': 1694.1206214595097}
Losses in iteration: 12 {'ner': 1511.02162952662}
Losses in iteration: 13 {'ner': 1301.605986669384}
Losses in iteration: 14 {'ner': 1277.415248008435}
Losses in iteration: 15 {'ner': 1126.4977596334538}
Losses in iteration: 16 {'ner': 1081.2835351016372}
Losses in iteration: 17 {'ner': 1116.588932958801}
Losses in iteration: 18 {'ner': 963.6739978024013}
Losses in iteration: 19 {'ner': 926.3754704474809}
Losses in iteration: 

Entities in 'Βότανα  Πεύκο: Θεραπευτικές ιδιότητες, οφέλη και χρήσειςΒόταναΠεύκο: Θ'
PLANT Πεύκο
PLANT Πεύκο
PLANT πεύκο
PLANT Πεύκο
PLANT πεύκο
PLANT πεύκο
PLANT πεύκου
PLANT πεύκων
PLANT καμφοράς
PLANT πεύκο
PLANT πεύκο
PLANT πεύκο
PLANT πεύκο
PLANT πεύκου
PLANT πεύκου
PLANT πεύκου
PLANT πεύκου
PLANT πεύκο
PLANT πορτοκάλι
ORGAN μύες
ORGAN ήπαρ
PLANT πεύκου
PLANT πεύκου
ORGAN συκώτι
PLANT κουκουναριού
PLANT πεύκου
PLANT πεύκου
ORGAN εντέρων
PLANT Πεύκο
PLANT πεύκα
ORGAN αρτηριών
PLANT πεύκου
ORGAN αρτηρίες
ORGAN τριχοειδή αγγεία
ORGAN εγκεφάλου
PLANT σταφυλιού
ORGAN καρδιά
ORGAN αγγείων
ORGAN καρδιάς
ORGAN εγκεφάλου
PLANT Πεύκο
Entities in 'Βότανα  Βότανα με οιστρογονική δράσηΒόταναΦυσική ΑντιμετώπισηΒότανα με'
PLANT καλέντουλα
PLANT γλυκόριζα
PLANT αγριοτριανταφυλλιά
PLANT χελιδόνιοπικραλίδα
PLANT δεντρολίβανο
PLANT κόκκινο τριφύλλι
PLANT βιβούρνο
PLANT Λυγαριά
PLANT λεβάντας
PLANT φασκομηλιάς
PLANT δυόσμου
PLANT λυγαριάς
PLANT Λυγαριά
PLANT Καναπίτσα
ORGAN μαστού
PLANT Maca
Entities

Entities in 'Φυσική Αντιμετώπιση  Μέλι: 11 συν 2 χρήσεις εντός και εκτός κουζίναςΦυ'
ORGAN δέρμα
PLANT λεμονιού
ORGAN έντερο
ORGAN στομάχι
ORGAN δέρμα
ORGAN δέρμα
ORGAN δέρμα
PLANT αλόη βέρα
PLANT Βάλσαμο
ORGAN μαλλιά
ORGAN μαλλιά
ORGAN μαλλιά
ORGAN μαλλιά
ORGAN δέρμα
PLANT λεβάντας
ORGAN δέρμα
PLANT πράσινο τσάι
PLANT Πράσινο τσάι
PLANT τζίντζερ
PLANT κανέλα
PLANT κανέλα
Entities in 'Βότανα  Οι βασικές ιδιότητες των γνωστότερων βοτάνωνΒόταναΟι βασικές ι'
PLANT τσουκνίδας
ORGAN συκώτι
PLANT φασκόμηλο
PLANT Φασκόμηλο
PLANT ευκάλυπτος
PLANT κυπαρίσσι
PLANT Ευκάλυπτος
PLANT χαμόμηλο
PLANT μελισσόχορτο
PLANT Μελισσόχορτο
PLANT θυμάρι
PLANT βάλσαμο
PLANT Θυμάρι
PLANT θυμαριού
PLANT τίλιο
PLANT Τίλιο
PLANT Φλαμούρι
PLANT ρίγανης
PLANT ρίγανη
PLANT μάραθος
PLANT Μάραθος
PLANT σκόρδο
PLANT Σκόρδο
PLANT σκόρδο
ORGAN στομάχι
PLANT πιπέρι
ORGAN καρδιάς
Entities in 'Βότανα  Χαμομήλι, Αντισηπτικό, αντιφλεγμονώδες και χαλαρωτικό-αγχολυτι'
PLANT βάλσαμο
ORGAN δέρμα
ORGAN νεύρα
ORGAN ματιών
ORGAN στομ

Entities in 'Βότανα  Βότανα και Μπαχαρικά: Πολύτιμη ασπίδα της υγείαςΒόταναΜπαχαρικ'
PLANT Πιπέρι
PLANT πιπέρι
PLANT πιπέρι
PLANT Μαϊντανός
PLANT μαϊντανός
PLANT Μαϊντανός
PLANT Ρίγανη
PLANT ρίγανη
PLANT ρίγανη
PLANT Γαρίφαλο
PLANT γαρίφαλο
PLANT γαρίφαλο
PLANT Κύμινο
PLANT κύμινο
PLANT Κύμινο
PLANT Τζίντζερ
PLANT τζίντζερ
ORGAN καρδιάς
ORGAN αρτηριών
PLANT τζίντζερ
PLANT Βασιλικός
PLANT βασιλικός
ORGAN εντέρου
PLANT Βασιλικός
PLANT Δεντρολίβανο
ORGAN εγκεφάλου
PLANT δεντρολίβανο
ORGAN εγκεφάλου
PLANT δεντρολίβανο
PLANT Κόλιανδρος
PLANT κόλιανδρου
PLANT Κόλιανδρος
PLANT Φασκόμηλο
PLANT φασκόμηλο
PLANT Φασκόμηλο
PLANT Θυμάρι
ORGAN εντέρου
PLANT θυμάρι
PLANT Θυμάρι
PLANT Τσίλι
PLANT Πιπέρι καγιένΤο
PLANT τσίλι
PLANT Κουρκουμάς
PLANT κουρκουμάς
ORGAN συκώτι
PLANT Κουρκουμάς
PLANT Σκόρδο
PLANT σκόρδο
PLANT Σκόρδο
PLANT Κανέλα
PLANT κανέλα
PLANT Κανέλα
ORGAN αρθρώσεων
ORGAN εντέρου
ORGAN εγκεφάλου
ORGAN καρδιάς
ORGAN ήπατος
Entities in 'Βότανα  Άνηθος-Anethum graveolens: Ιδιότητες, οφέλη κα

Entities in 'Βότανα  Καρυδιά: Φαρμακευτικές ιδιότητες, Χρήσεις και ΚαλλιέργειαΒόταν'
PLANT Καρυδιά
PLANT Καρυδιά
PLANT καρυδιά
PLANT καρυδιά
PLANT καρύδι
PLANT καρυδιάς
PLANT καρυδιάς
ORGAN ματιών
ORGAN μήτρας
ORGAN αμυγδαλών
ORGAN οστών
ORGAN νύχια
ORGAN αρθρώσεων
ORGAN αμυγδαλών
ORGAN δέρματος
PLANT καρυδιού
PLANT καρύδια
PLANT σκόρδου
PLANT καρυδιά
PLANT καρυδιά
PLANT καρυδιές
PLANT καρυδιάς
PLANT καρυδιά
PLANT καρυδιά
PLANT καρυδιές
PLANT καρυδιά
PLANT καρυδιάς
PLANT καρύδια
PLANT καρύδια
PLANT καρυδιών
PLANT καρυδιά
PLANT καρύδια
PLANT καρυδιάς
ORGAN στομάχι
PLANT καρυδιών
PLANT καρυδιάς
PLANT καρυδιού
PLANT καρύδια
PLANT καρύδια
PLANT καρυδιά
PLANT καρυδιών
PLANT καρύδια
PLANT καρυδιάς
PLANT καρυδιάς
ORGAN δέρματα
ORGAN μαλλιών
PLANT καρυδιάς
PLANT καρυδιών
PLANT καρύδια
PLANT καρυδιού
PLANT σταφύλια
PLANT καρυδιάς
PLANT καρυδιάς
PLANT καρυδιάς
PLANT καρυδιών
ORGAN ματιών
PLANT μπελαντόνα
PLANT λάβδανο
ORGAN μαλλιών
PLANT καρύδι
PLANT καρυδιά
PLANT καρυδιάς
PLANT καρυδιές
PLANT τ

...

Entities in 'Βότανα  Βότανα και φυσική αντιμετώπιση Μυοσκελετικών προβλημάτωνΒότανα'
PLANT Capsicum
PLANT Cimicifuga racemosa
PLANT ginseng
ORGAN νεφρών
ORGAN χόνδρος
PLANT Φραγκοστάφυλο
ORGAN δέρματος
ORGAN ωοθηκών
ORGAN όρχεων
PLANT Φυτολάκκα
PLANT Σέλινο
PLANT σέλινο
ORGAN αρθρώσεων
PLANT σέλινου
PLANT σέλινου
ORGAN αρθρώσεις
ORGAN στομάχου
PLANT Φιλιπέντουλα
PLANT Ιτιά
ORGAN αρθρώσεων
ORGAN καρδιά
PLANT Αρπαγόφυτο
ORGAN αρθρώσεων
ORGAN αγγείων
PLANT Αρπαγόφυτο
ORGAN γλώσσα
ORGAN συκωτιού
PLANT Μηνύανθος
PLANT Αγριμόνιο
ORGAN μυς
ORGAN στομάχι
ORGAN έντερα
PLANT Cimicifuga racemosa
PLANT Black cohosh
ORGAN υποθάλαμο
ORGAN υπόφυση
ORGAN μήτρα
ORGAN μήτρας
PLANT Τσιμιτσιφούγκα
PLANT Black Cohosh
PLANT άρνικα
PLANT άρνικα
PLANT άρνικα
PLANT Άρνικα
ORGAN μήτρας
ORGAN μήτρας
ORGAN συκωτιού
ORGAN βλεννογόνου
PLANT Σύμφυτο
PLANT capsicum
PLANT Λομπέλια
ORGAN πνεύμονες
ORGAN αρθρώσεις
ORGAN αρθρώσεις
ORGAN χόνδρου
ORGAN αρθρώσεις
Entities in 'Βότανα  Γλιστρίδα-Αντράκλα, ζιζάνιο που πετάνε α

Entities in 'Βότανα  Τα είδη του τσαγιού και οι ιδιότητές τουςΒόταναΤα είδη του τσα'
PLANT μαύρο τσάι
PLANT θρύλος
PLANT μαύρο τσάι
PLANT τσάι oolong
PLANT Camellia sinensis
PLANT μαύρο τσάι
PLANT κόκκινο τσάι
ORGAN γλώσσα
ORGAN προστάτη
ORGAN στομάχου
ORGAN εντέρου
PLANT λευκό τσάι
PLANT λευκό τσάι
ORGAN δοντιών
PLANT λευκό τσάι
PLANT λευκό τσάι
ORGAN εγκέφαλο
PLANT λευκό τσάι
ORGAN στομάχου
ORGAN μαστού
PLANT Μανιτάρια
PLANT πράσινο τσάι
ORGAN μαστού
ORGAN οστών
PLANT Πράσινο τσάι
PLANT κόκκινο τσάι
PLANT μαύρο τσάι
PLANT μαύρο τσάι
PLANT μαύρο τσάι
PLANT μαύρο τσάι
PLANT πράσινο τσάι
PLANT κόκκινο τσάι
PLANT πράσινο τσάι
PLANT μαύρο τσάι
PLANT τσάι του βουνού
PLANT Τσάι του βουνού
PLANT Τσάι του βουνού
PLANT Τσάι Βουνού
Entities in 'Βότανα  Σέλινο και λουτεολίνη για φλεγμονές, άνοια, ουρική αρθρίτιδα, '
PLANT Σέλινο
PLANT Σέλινο
PLANT σέλινο
PLANT πιπεριά
PLANT μαϊντανό
PLANT θυμάρι
PLANT βασιλικό
PLANT δυόσμο
PLANT Σέλινο
PLANT σέλινο
PLANT πιπεριές
ORGAN εγκεφάλου
ORGAN εγκεφάλου


Entities in 'Τσάι ιβίσκου, άλλη μια φυσική ασπίδα για την υγείαΟ ιβίσκος είναι ένα '
PLANT ιβίσκου
PLANT ιβίσκος
PLANT ιβίσκος
PLANT ιβίσκος
ORGAN δέρμα
PLANT ιβίσκος
PLANT ιβίσκου
PLANT ιβίσκου
ORGAN ήπαρ
Entities in 'Βότανα  Πανσές, για εκζέματα, ψωρίαση, ακμή, ρευματισμούς, κυστίτιδες '
PLANT Πανσές
PLANT Πανσές
PLANT πανσές
PLANT πανσές
ORGAN δέρματος
ORGAN καρδιάς
ORGAN δέρματος
PLANT πανσές
ORGAN δέρμα
ORGAN πνεύμονες
ORGAN δέρματος
ORGAN δέρμα
ORGAN τριχοειδών αγγείων
PLANT πανσές
ORGAN φλέβες
ORGAN νεφρών
PLANT Βήχιο
PLANT Τριφύλλι
PLANT Τσουκνίδα
PLANT Γάλιο
PLANT Αγριάδα
PLANT πανσές
ORGAN μάτι
PLANT λάχανα
PLANT Ροδοπούλου
Entities in 'Βότανα  Βαλσαμόχορτο και Βαλσαμόλαδο: Χρήσεις και ΟφέληΒόταναΘεραπευτι'
PLANT Βαλσαμόχορτο
PLANT Βαλσαμόχορτο
PLANT βαλσαμόχορτο
PLANT σπαθόχορτο
PLANT Βάλσαμο
PLANT σπαθόχορτο
ORGAN στομαχιού
ORGAN πνευμόνων
ORGAN συκωτιού
PLANT κισσό
PLANT σαλέπι
PLANT σταφυλιών
PLANT βάλσαμου
PLANT βάλσαμο
PLANT σπαθόχορτο
PLANT σπαθόχορτου
ORGAN δέρματος
O

Entities in 'Βότανα  Γάλιο-απαρίνη ή κολλιτσίδα ιδανικό διεγερτικό του λεμφικού συσ'
PLANT κολλιτσίδα
PLANT κολλιτσίδα
PLANT κολλιτσίδα
PLANT κολλιτσίδα
PLANT γάλιο
PLANT γάλιο
PLANT σπανάκι
PLANT γάλιο
PLANT κολλιτσίδα
PLANT γάλιο
ORGAN καρδιάς
ORGAN μυαλού
PLANT γάλιο
PLANT γάλιο
ORGAN δέρματος
ORGAN δέρματος
ORGAN δέρματος
PLANT γάλιο
PLANT καλέντουλα
PLANT γάλιο
ORGAN αγγείων
PLANT κολλιτσίδα
PLANT γάλιο
PLANT φυτολάκκα
PLANT καλέντουλα
PLANT κολλιτσίδα
ORGAN δέρμα
ORGAN οστά
PLANT φιτίλι
PLANT κολλιτσίδας
PLANT χρειαζόμαστεφύλλα
PLANT κολλιτσίδα
PLANT κολλιτσίδα
PLANT κολλιτσίδα
Entities in 'Βότανα  Τσάι του βουνού: Εύγευστο ρόφημα, με πολλές ευεργετικές δράσει'
PLANT Τσάι του βουνού
PLANT δράσειςΒόταναΤσάι του βουνού
PLANT Τσάι του βουνού
PLANT Τσάι του βουνού
PLANT τσάι του βουνού
PLANT τσάι του βουνού
ORGAN εγκέφαλο
PLANT λεμονιού
PLANT κανέλα
PLANT λεμόνι
PLANT τσάι του βουνού
PLANT Τσάι του βουνού
PLANT τσάι του βουνού
PLANT Τσάι Βουνού
PLANT τσάι του βουνού
Entities in 'Βότα

Entities in 'Βότανα  Το λεξικό των βοτάνων – Όλα από ΧΒόταναΓιατροσόφιαΤο λεξικό τω'
PLANT χαμομηλιού
PLANT χαμομηλιού
ORGAN μαλλιά
PLANT χαμομηλιού
PLANT Χελιδόνιο
ORGAN συκωτιού
ORGAN αυτιά
ORGAN ματιών
PLANT χρένο
PLANT Χρυσάνθεμο
PLANT αψιθιά
PLANT καμφορά
PLANT λεμόνι
PLANT καρότο
PLANT λάχανο
PLANT κρεμμύδι
PLANT λεμόνι
PLANT σέλινο
PLANT Αγιάγκαθο
PLANT αγκινάρα
PLANT άκορος
PLANT ασπερούλα
PLANT γλυκόριζα
PLANT δενδρολίβανο
PLANT ευώνυμο
PLANT καλαμπόκι
PLANT καρότο
PLANT κολλιτσίδα
PLANT κρεμμύδι
PLANT λάππα
PLANT μέντα
PLANT σκόρδο
PLANT φλισκούνι
PLANT Αγκινάρα
PLANT γλυκόριζα
PLANT πικραλίδα
PLANT σουσάμι
PLANT Αγγουριά
PLANT αγριάδα
PLANT αγκινάρα
PLANT δυόσμος
PLANT κρεμμύδι
PLANT μέντα
PLANT πικραλίδα
PLANT σέλινο
PLANT σέλινο
PLANT Αγκινάρα
PLANT αγκινάρες
PLANT αγκινάρες
ORGAN συκωτιού
PLANT Ραδίκι
ORGAN συκώτι
ORGAN συκώτι
PLANT ραδίκια
PLANT ραδικιού
PLANT Καρότο
PLANT καρότο
ORGAN εντέρου
PLANT καρότο
PLANT καρότα
PLANT ελιάς
PLANT ελιάς
PLANT κρεμμύδι
PLANT κρεμμύδ